<h1><center>DS420: Homework 5 Spark Streaming


## Goal
In this homework, your goal to setup an end-to-end data streaming application to collect data from Twitter and analyze the data with Spark Streaming and Spark SQL. 

Firstly, we need to setup a Developer API acocunt with Twitter and create an application to get credentials. 
    
Once you have that you then need to complete the `hw5_twitterKafkaProducer.py` program in order to setup the twitter stream producer. You can verify whether the producer is up running by starting a kafka-console-consumer on the side to monitor. 

Lastly, you need to finish this notebook to setup a Spark Streaming consumer, analyze the stream in real-time, and finally visualize some of the tweeter stream activities with Pandas. Keep your producer running while working on this notebook. 

Let's get started!

In [1]:
import findspark
findspark.init('/opt/spark')

## Q1: Create a Spark session, and named it as `hw5_xxx`, where `xxx` is your last name.

In [2]:
from pyspark.sql import SparkSession


In [3]:
# Just change the appName but don't modify the configurations!

spark = SparkSession.builder.appName('sparkStreaming_PE6_moynihan')          \
                    .config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1')\
                    .config('spark.jars.packages','org.apache.kafka:kafka-clients:2.4.1')\
                    .getOrCreate()


22/05/09 19:53:44 WARN Utils: Your hostname, GPUServer resolves to a loopback address: 127.0.1.1; using 10.4.10.8 instead (on interface enp2s0)
22/05/09 19:53:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/moynihanl/.ivy2/cache
The jars for the packages stored in: /home/moynihanl/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark-3.0.1/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-390f72f1-0a89-461b-9153-de1c346c1c4d;1.0
	confs: [default]
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.3-1 in central
	found org.lz4#lz4-java;1.6.0 in central
	found org.xerial.snappy#snappy-java;1.1.7.3 in central
	found org.slf4j#slf4j-api;1.7.28 in central
:: resolution report :: resolve 116ms :: artifacts dl 3ms
	:: modules in use:
	com.github.luben#zstd-jn

In [4]:
# Read data from kafka into a streaming DataFrame
# Ensure to supply the bootstrap server and the channel;
# and include the timestamp as you read it. 

streaming = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "ds420-moynihan") \
  .option('includeTimestamp','True')\
  .load()


In [5]:
# Select only timestamp and value columns. Cast value column into String type.
streaming = streaming.selectExpr( "CAST(value AS STRING)", 'timestamp')



In [6]:
streaming.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



## Q2: Run a simple query to show the raw data of the `value` column.

In [7]:
# Grab the `value` column only from the streaming DataFrame into rawValue
rawValue = streaming.select('value')



In [8]:
# Start a streaming query with rawValue

valueQuery = (rawValue.writeStream
             .format('memory')
             .queryName('valueQuery')
             .start())




22/05/09 19:53:49 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a668137a-af81-4289-ab1f-230ac981ab56. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [9]:
# Poll the results every 5 seconds and show the first ten records
# Run this for 5 iterations. 

import time



for x in range(10):
    df = spark.sql('SELECT * FROM valueQuery')
    df.show(10)
    time.sleep(1)



+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+



In [10]:
# Stop the query once it's done. 

valueQuery.stop()

## Q3: Create schema for the raw data with the hint from the printed results. 

In [11]:
from pyspark.sql.types import *

# Create the dataSchema with StructType()

dataSchema = dataSchema = StructType([ \
    StructField("id",StringType(),True), \
    StructField("user",StringType(),True), \
    StructField("follower",IntegerType(),True), \
    StructField("friends", IntegerType(), True), \
    StructField("tweet", StringType(), True), \
    StructField("hashtag", IntegerType(), True), \
    StructField("ts", TimestampType(),True), \
    StructField("retweet",IntegerType(),True), \
    StructField("favorite",IntegerType(),True) \
 
  ])



In [12]:
# Show the schema, use this as the hint to finish the question above

print(dataSchema)

StructType(List(StructField(id,StringType,true),StructField(user,StringType,true),StructField(follower,IntegerType,true),StructField(friends,IntegerType,true),StructField(tweet,StringType,true),StructField(hashtag,IntegerType,true),StructField(ts,TimestampType,true),StructField(retweet,IntegerType,true),StructField(favorite,IntegerType,true)))


#### Create a new column for the `streaming` DataFrame named as `json`. Use `from_json` function to parse the string in `value` column into `json` column. Save the resulting DataFrame into `df`.
`from_json` docs: [link](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.from_json)

In [13]:
from pyspark.sql.functions import *

df = streaming.withColumn('json', from_json('value', dataSchema))



In [14]:
# Checkout the schema of df. 
df.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- json: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- user: string (nullable = true)
 |    |-- follower: integer (nullable = true)
 |    |-- friends: integer (nullable = true)
 |    |-- tweet: string (nullable = true)
 |    |-- hashtag: integer (nullable = true)
 |    |-- ts: timestamp (nullable = true)
 |    |-- retweet: integer (nullable = true)
 |    |-- favorite: integer (nullable = true)



## Q4: Generate new columns with the sub-fields from the `json` column.

#### Hint1: once the dataSchema is contructed, we can use the `.name` and `.dataType` properties to find its field name and field data type, respectively. Run the code example below.

In [15]:
for idx, field in enumerate(dataSchema): 
    print(idx, field.name, field.dataType)

0 id StringType
1 user StringType
2 follower IntegerType
3 friends IntegerType
4 tweet StringType
5 hashtag IntegerType
6 ts TimestampType
7 retweet IntegerType
8 favorite IntegerType


#### Hint2: notice you can use dot operator to index into the nested structure. Run the code below:

In [16]:
df.select('json.follower')

DataFrame[follower: int]

#### Also works with bracket notation.

In [17]:
df['json.follower']

Column<b'json.follower AS `follower`'>

#### Now, create new columns with all the sub-fields from the `json` column by using a for loop. 

In [18]:
for idx, field in enumerate(dataSchema): 
    a = df.withColumn(field.name, df['json.' + field.name])
    df=a

In [19]:
# Checkout the schema after the update
df.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- json: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- user: string (nullable = true)
 |    |-- follower: integer (nullable = true)
 |    |-- friends: integer (nullable = true)
 |    |-- tweet: string (nullable = true)
 |    |-- hashtag: integer (nullable = true)
 |    |-- ts: timestamp (nullable = true)
 |    |-- retweet: integer (nullable = true)
 |    |-- favorite: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- follower: integer (nullable = true)
 |-- friends: integer (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtag: integer (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- retweet: integer (nullable = true)
 |-- favorite: integer (nullable = true)



#### Finally, clean the `df` DataFrame so that it only contains the sub-fileds from `json` column, and the `timestamp` column

In [20]:

df = df.select(['user','id', 'follower','tweet', 'hashtag', 'ts', 'retweet', 'favorite','timestamp', 'friends'])


In [21]:
df.printSchema()

root
 |-- user: string (nullable = true)
 |-- id: string (nullable = true)
 |-- follower: integer (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtag: integer (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- retweet: integer (nullable = true)
 |-- favorite: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- friends: integer (nullable = true)



## Q5: Run a query to find the occurences of each number of 'retweet'. 

In [22]:
retweetCount =df.groupBy('retweet').count()



In [23]:
retweetQuery = retweetCount.writeStream \
                  .outputMode('complete') \
                  .format('memory') \
                  .option('truncate', 'false') \
                  .queryName('retweetQuery') \
                  .start()




22/05/09 19:54:11 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a8b08088-2d5d-48ec-ab34-8473e19483a4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [24]:
for i in range(10):
    df_ = spark.sql('SELECT * FROM retweetQuery')
    df_.show(10)
    time.sleep(2)


+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



## Q6: Run a windowed query to find the number of occurences for each number of 'follower'. Use window size as 20 seconds, and sliding frequency as 10 seconds.

In [25]:
from pyspark.sql.functions import window


In [26]:
followerCount = df.groupBy(
    window(df['timestamp'],'1 minutes','30 seconds'),
    df['follower']
).count().orderBy('window')




In [27]:
followerQuery = (followerCount.writeStream
                  .outputMode('complete')
                  .format('memory')
                  .queryName('followerQuery')
                  .start()
)


22/05/09 19:54:31 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1badf097-2b47-42df-a28d-0c3b3c9398cc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [28]:

for i in range(10):
    df_ = spark.sql('SELECT * FROM retweetQuery')
    df_.show(10)
    time.sleep(1)





+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+

+-------+-----+
|retweet|count|
+-------+-----+
+-------+-----+



## Q7: Analyze the sentiment of the tweets by using the TextBlob function.

TextBlob help doc: [link](https://textblob.readthedocs.io/en/dev/)

#### First, write a function that takes in a text input, and returns a sentiment of the input from the following array: ['negative', 'neutral', 'positive']

In [29]:
from textblob import TextBlob

def get_sentiment(text):
    textblob = TextBlob(text).sentiment.polarity
    if textblob < 0:
        return 'negative'
    elif textblob == 0:
        return 'neutral'
    elif textblob > 0:
        return 'positive'
    
    
    

#### Second, use a UDF function to generate a new column for the streaming DataFrame `df`.

In [30]:
from pyspark.sql.functions import udf

sentiment_udf = udf(get_sentiment, StringType())


df = df.withColumn('sentiment', sentiment_udf(df.tweet))



In [31]:
df.printSchema()

root
 |-- user: string (nullable = true)
 |-- id: string (nullable = true)
 |-- follower: integer (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtag: integer (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- retweet: integer (nullable = true)
 |-- favorite: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- friends: integer (nullable = true)
 |-- sentiment: string (nullable = true)



#### Lastly, run a query to find the count for each kind of sentiment. 

In [32]:
sentimentCount = df.groupBy('sentiment').count()



In [33]:
sentimentQuery = sentimentCount.writeStream \
                  .outputMode('complete') \
                  .format('memory') \
                  .queryName('sentimentQuery') \
                  .start()




22/05/09 19:54:42 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-44c6a297-d5a1-4a5f-bd47-7a802514bbd7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [34]:
# Can also use an infinitely loop instead to poll the results

for i in range(10):
    df = spark.sql('SELECT * FROM sentimentQuery')
    df.show(10)
    time.sleep(2)

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+



+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+

+---------+-----+
|sentiment|count|
+---------+-----+
+---------+-----+



## Q8: Show the sentiment analysis on a barplot dashboard.

In [35]:
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [38]:
for i in range(10):
    df = spark.sql('SELECT * FROM sentimentQuery')
    df_pd = df.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize = (10,8))
    sns.barplot(data = df_pd, y='sentiment', x='count')
    plt.show()
    time.sleep(1)

ValueError: min() arg is an empty sequence

<Figure size 720x576 with 0 Axes>

In [37]:
# Stop all queries
retweetQuery.stop()
followerQuery.stop()
sentimentQuery.stop()

## Great Job!